# Touches on the disc

Ultimate is a game of possession, similar to American football or European soccer. Similar to the latter, all players usually touch the disc while their team moves it around the field. And, similar to both sports, players tend to have skewed statistics on their usage percentage -- some players are high volume and others low.

Additionally, in the mixed division of the sport their is an asymmetry on the field. One gender will have 4 players and the other gender will have 3 (all on the same team, and the other team mirrors).

If both genders were on the field symmetrically, one would expect the usage percentage to be split between them (1:1 ratio) with all other factors being equal. Alternatively, if all positions were equivalent then one would expect the usage percent to be split according to a 4:3 ratio. Factoring in asymmetric positions, we turn to numerical simulations to find the expected usage ratios.

In [1]:
# First import a bunch of libraries
import numpy as np
import numpy.random as rand
import matplotlib.pyplot
%matplotlib inline

## Player object

Create a player object, to which we can assign it an ID number (0-6 for each player), gender, position, and a passing function. The passing function is an arbitrary function that takes in a gender, position, and team information and determines how the player will tend to throw to their teammates.

This player object+passing function scheme can work with any gender/position configuration. For instance on an open or womens team, the gender won't influence the passing ratio at all. Conversely, by letting the passing function be completely general, then one could design passing functions to test for systemic trends with a parametric model (this would be well beyond the scope of this notebook :-) ).

In [14]:
class Player(object):
    def __init__(self, ID, gender, position, passing_function):
        """Constructor used to create a player."""
        self.ID               = ID
        self.gender           = gender
        self.position         = position
        self.passing_function = passing_function

    def __str__(self):
        """Diagnostic for printing."""
        return "Player ID %d:\n\tGen: %s\n\tPos: %s\n\tPassing function: %s"%(self.ID,self.gender,self.position,self.passing_function.__name__)
        
    def make_pass(self, the_team):
        """A wrapper to the passing function, that takes in the team.
        Returns the ID number of a teammate that they have passed to."""
        return self.passing_function(self.ID, self.gender, self.position, the_team)

In [15]:
#Test that the player object is working
def fake_passing_function(ID, gender, position, the_team, *args):
    """Always pass to player 0."""
    return 0

Player1 = Player(1, "M", "H", fake_passing_function)
print str(Player1)
print "Pass result: ",Player1.make_pass(None)

Player ID 1:
	Gen: M
	Pos: H
	Passing function: fake_passing_function
Pass result:  0


## Team object

The team object is simple: it is a collection of players. This collection is used to run individual players' passing functions. It takes in an ordered list of genders, an ordered list of positions, and an ordered list of passing functions.

In [18]:
class Team(object):
    def __init__(self, gender_list, position_list, pf_list):
        """Constructor for the team."""
        if len(gender_list) != len(position_list) != len(pf_list):
            raise Exception("Must supply all information for all players.")
        N_players = len(gender_list)
        self.N_players = N_players
        self.players = [Player(i, gender_list[i], position_list[i], pf_list[i]) for i in range(N_players)]
        
    def __str__(self):
        out = str(self.players[0])
        for i in range(1,self.N_players):
            out += "\n"+str(self.players[i])
        return out

In [21]:
# Test that the team objects work.
gens = ["m" for i in range(7)]
poss = ["h" for i in range(7)]
pfs = [fake_passing_function for i in range(7)]
Team1 = Team(gens, poss, pfs)
print Team1

Player ID 0:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 1:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 2:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 3:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 4:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 5:
	Gen: m
	Pos: h
	Passing function: fake_passing_function
Player ID 6:
	Gen: m
	Pos: h
	Passing function: fake_passing_function


## Passing functions
Now we need to create some passing functions. The most naive passing function will have a player randomly pass to any player besides themselves.

In [ ]:
def naive_passing_function(ID, gender, position, the_team, *args):
    N = the_team.N_players
    